In [2]:
import os
os.chdir("../src/expB/")

# Distance Learner for Adversarial Examples

Steps:

1. Train distance function with multiple spheres.
2. Construct a classifier which classifies based on the closest distance to the spheres.
3. Compare this to a standard classifier which learns the classification task directly (instead of learning distance functions).
4. Adversarial attacks can be detected as points which are at least a threshold distance from the closest manifold.
5. Compare adversarial robustness via above method and current adversarial robustness methods on classifiers (e.g. Madry et.al’s adversarial training).
6. Run above on different synthetic manifolds.Run above on real datasets: MNIST, Cifar.

## 1. Train distance function with multiple spheres.

### 1.1 Generate the dataset




In [ ]:
%run spheres_v2.py --twospheres --config data_configs/2sphere_expC_config.json --save_dir /azuredrive/datasets/expC/2spheres






### 1.2 Training a Distance Regressor on the generated 2spheres dataset

In [2]:
%run -i learn_mfld_distance.py --train --specs ./expt_specs/specs_distlearn_2spheres_expC_31052021.json




100%|██████████| 500/500 [58:46<00:00,  7.05s/it]


## 2. Construct a classifier which classifies based on the closest distance to the spheres.

### 2.1 Compute and store logits from Distance Regressor

In [3]:
%run -i learn_mfld_distance.py --test --specs ./expt_specs/specs_distlearn_2spheres_expC_test_01062021.json




name: train
MSE for the learned distances: 8.024667e-06
MSE for the learned distances (on-manifold): 8.024749e-06
name: val
MSE for the learned distances: 9.055453e-06
MSE for the learned distances (on-manifold): 9.055488e-06
name: test
MSE for the learned distances: 8.862155e-06
MSE for the learned distances (on-manifold): 8.862197e-06


.  ..  logits.pt  specs.json  targets.pt


'/azuredrive/dumps/expC_dist_learner_for_adv_ex/dist_learner/test_MLPwithNormalization_model{hidden_sizes=512x4,sigmoid_last,batch_norm}_data{2spheres_in_unit_cube;n=100;k=2}/31052021-192633'

### 2.2 Compute optimal threshold using training set

## 3. Compare this to a standard classifier which learns the classification task directly (instead of learning distance functions).

### 3.1 Train a standard classifier on the same dataset

In [2]:
%run -i learn_mfld_distance.py --train --specs ./expt_specs/specs_stdclf_2spheres_expC_31052021.json




100%|██████████| 500/500 [43:08<00:00,  5.18s/it]


### 3.2 Compute and store logits for standard classification